<a href="https://colab.research.google.com/github/Jojowaranyu/weatherwise-Waranyu-Bancherdvanich/blob/main/starter_notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🌦️ WeatherWise – Starter Notebook

Welcome to your **WeatherWise** project notebook! This scaffold is designed to help you build your weather advisor app using Python, visualisations, and AI-enhanced development.

---

📄 **Full Assignment Specification**  
See [`ASSIGNMENT.md`](ASSIGNMENT.md) or check the LMS for full details.

📝 **Quick Refresher**  
A one-page summary is available in [`resources/assignment-summary.md`](resources/assignment-summary.md).

---

🧠 **This Notebook Structure is Optional**  
You’re encouraged to reorganise, rename sections, or remove scaffold cells if you prefer — as long as your final version meets the requirements.

✅ You may delete this note before submission.



## 🧰 Setup and Imports

This section imports commonly used packages and installs any additional tools used in the project.

- You may not need all of these unless you're using specific features (e.g. visualisations, advanced prompting).
- The notebook assumes the following packages are **pre-installed** in the provided environment or installable via pip:
  - `requests`, `matplotlib`, `pyinputplus`
  - `fetch-my-weather` (for accessing weather data easily)
  - `hands-on-ai` (for AI logging, comparisons, or prompting tools)

If you're running this notebook in **Google Colab**, uncomment the following lines to install the required packages.


In [202]:
# 🧪 Download packages
!pip install hands-on-ai pyinputplus

In [203]:
import os

os.environ['HANDS_ON_AI_SERVER'] = 'http://ollama.serveur.au'
os.environ['HANDS_ON_AI_MODEL'] = 'granite3.2'

## 📦 Setup and Configuration
Import required packages and setup environment.

In [204]:
import requests
import matplotlib.pyplot as plt
import pyinputplus as pyip
# ✅ Import after installing (if needed)
from hands_on_ai.chat import get_response

# Add any other setup code here

# Core libraries
import os
import re
from typing import Dict, Any, List, Optional, Tuple, Union
import requests
import matplotlib.pyplot as plt

os.environ["OPENWEATHER_API_KEY"] = "3a10e176bbe73766db6e4d09a4bd96d0"  # Demo key, get your own at https://openweathermap.org/api


# pyinputplus is preferred for menus, but guard if missing in some environments
try:
	import pyinputplus as pyip
except Exception:
	pyip = None  # UI cell will handle None gracefully

# Fetch weather via the teaching helper library
try:
	import fetch_my_weather as fmw
except Exception as e:
	fmw = None
	print("Warning: fetch_my_weather not installed. You can install it in Colab by running the optional pip cell above.")

# Global state used by UI/demo (kept minimal)
_LAST_LOCATION: Optional[str] = None
_LAST_FORECAST_DAYS: int = 5
_LAST_WEATHER: Optional[Dict[str, Any]] = None

# Helper for safe matplotlib usage in notebooks
plt.rcParams.update({
	"figure.figsize": (8, 4),
	"axes.grid": True,
})

## 🌤️ Weather Data Functions

In [205]:
# Define get_weather_data() function here
from typing import TypedDict

class ParsedWeather(TypedDict, total=False):
	location: str
	forecast_days: int
	error: Optional[str]
	current: Dict[str, Any]
	daily: List[Dict[str, Any]]


def _normalize_fmw_json(model_or_dict: Any) -> Dict[str, Any]:
	"""
	Convert fetch_my_weather WeatherResponse (Pydantic) or raw dict to a plain dict.
	This keeps only the keys we care about for the assignment to simplify downstream code.
	"""
	# If it's a pydantic model, use model_dump if available
	if hasattr(model_or_dict, "model_dump"):
		data = model_or_dict.model_dump()  # type: ignore[attr-defined]
	else:
		data = dict(model_or_dict)
	return data

def openWeather_getWeather(city: str, units: str = "metric"):
	api_key = os.getenv("OPENWEATHER_API_KEY")
	if not api_key:
		return None
	try:
		CITY = city
		UNITS = units  # metric for °C or "imperial" for °F
		# Step 1: Call the 5-day forecast API
		url = f"https://api.openweathermap.org/data/2.5/forecast?q={CITY}&appid={api_key}&units={UNITS}"
		response = requests.get(url)
		data = response.json()
		return data
	except Exception as e:
		print(f"Error fetching weather data: {e}")
		return None


def get_weather_data(city: str, forecast_days: int = 5):
	"""
	Retrieve weather data for a specified location.

	Args:
		location (str): City or location name
		forecast_days (int): Number of days to forecast (1-5)

	Returns:
		dict: Weather data including current conditions and forecast
	"""
	try:
		response = openWeather_getWeather(city=city, units="metric")
		return response
	except Exception as e:
		response["error"] = f"Failed to fetch weather data: {e}"
		return response

## 📊 Visualisation Functions

In [206]:
# Define create_temperature_visualisation() and create_precipitation_visualisation() here
from datetime import datetime
from typing import List, Dict, Any

def _extract_day_labels(daily: List[Dict[str, Any]]) -> List[str]:
        labels: List[str] = []
        for d in daily:
                date_str = d.get("date") or d.get("astronomy", [{}])[0].get("sunrise")
                try:
                        labels.append(datetime.strptime(date_str, "%Y-%m-%d").strftime("%a %d"))
                except Exception:
                        labels.append(str(date_str))
        return labels

def get_temps(data):
        daily_temps = {}

        for item in data["list"]:
                dt_txt = item["dt_txt"] # e.g., '2025-10-16 03:00:00'
                date_str = dt_txt.split(" ")[0] # Just the date part
                temp = item["main"]["temp"]

                if date_str not in daily_temps:
                        daily_temps[date_str] = []

                daily_temps[date_str].append(temp)

        return daily_temps


def create_temperature_visualisation(weather_data: Dict[str, Any], num_days: int = 5, output_type: str = 'display'):
        """
        Create visualisation of temperature data using average daily temperatures.

        Args:
                weather_data (dict): The processed weather data
                num_days (int): The number of days to display in the chart.
                output_type (str): Either 'display' to show in notebook or 'figure' to return the figure

        Returns:
                If output_type is 'figure', returns the matplotlib figure object
                Otherwise, displays the visualisation in the notebook
        """

        # get daily forecast data
        daily = weather_data.get("list", [])
        if not daily:
                print("No daily forecast data available to plot temperatures.")
                return None

        daily_temps = get_temps(weather_data)
        # Slice the data to include only the requested number of days
        sliced_dates = list(daily_temps.keys())[:num_days]
        sliced_temps = [sum(daily_temps[date]) / len(daily_temps[date]) if daily_temps[date] else 0 for date in sliced_dates]


        fig, ax = plt.subplots()
        ax.plot(sliced_dates, sliced_temps, marker='o', label='Average Temp (°C)')
        ax.set_title(f"Daily Average Temperature Forecast for {num_days} Days")
        ax.set_xlabel("Day")
        ax.set_ylabel("Temperature (°C)")
        ax.legend()
        plt.xticks(rotation=45)
        plt.tight_layout()

        if output_type == 'figure':
                return fig
        else:
                plt.show()
                return None

In [207]:

def create_precipitation_visualisation(weather_data: Dict[str, Any], output_type: str = 'display'):
	"""
	Create visualisation of precipitation data.

	Args:
		weather_data (dict): The processed weather data
		output_type (str): Either 'display' to show in notebook or 'figure' to return the figure

	Returns:
		If output_type is 'figure', returns the matplotlib figure object
		Otherwise, displays the visualisation in the notebook
	"""
	daily = weather_data.get("daily", [])
	if not daily:
		print("No daily forecast data available to plot precipitation.")
		return None

	# Some wttr.in data uses hourly precipitation; daily often contains total precipitation in mm
	# We'll use total precipitation in mm if available; otherwise, estimate from hourly
	totals: List[float] = []
	labels = _extract_day_labels(daily)

	for d in daily:
		mm = d.get("totalPrecipMM")
		if mm is None and d.get("hourly"):
			try:
				mm = sum(float(h.get("precipMM") or 0.0) for h in d["hourly"])  # type: ignore[index]
			except Exception:
				mm = 0.0
		try:
			totals.append(float(mm))
		except Exception:
			totals.append(0.0)

	fig, ax = plt.subplots()
	ax.bar(labels, totals, color="#4e79a7")
	ax.set_title("Daily Total Precipitation")
	ax.set_xlabel("Day")
	ax.set_ylabel("Precipitation (mm)")
	plt.tight_layout()

	if output_type == 'figure':
		return fig
	else:
		plt.show()
		return None

## 🤖 Natural Language Processing

In [208]:
from typing import Dict, Any

def parse_weather_question(question: str) -> Dict[str, Any]:
        """
        Parse a natural language weather question.

        Args:
                question (str): User's weather-related question

        Returns:
                dict: Extracted information including location, time period, and weather attribute
        """
        if not question or not isinstance(question, str):
                return {"error": "Invalid question provided."}

        # Basic implementation to return a dummy structure
        return {
                "raw": question,
                "location": None,
                "temperature": None,
                "uv_index": None,
                "weather_condition": None
        }

## 🧭 User Interface

In [224]:
# Define menu functions using pyinputplus or ipywidgets here
import json
def get_temps(data):
        daily_temps = {}

        for item in data["list"]:
                dt_txt = item["dt_txt"] # e.g., '2025-10-16 03:00:00'
                date_str = dt_txt.split(" ")[0] # Just the date part
                temp = item["main"]["temp"]

                if date_str not in daily_temps:
                        daily_temps[date_str] = []

                daily_temps[date_str].append(temp)

        return daily_temps

def run_console_menu():
        """Simple console menu using pyinputplus if available, else fallback to input()."""
        global _LAST_LOCATION, _LAST_FORECAST_DAYS, _LAST_WEATHER

        def ask(prompt: str, choices: Optional[List[str]] = None) -> str:
                if pyip and choices:
                        return pyip.inputMenu(choices, prompt=prompt + "\n", numbered=True)
                elif pyip:
                        return pyip.inputStr(prompt)
                else:
                        # basic fallback
                        return input(prompt + (f" ({'/'.join(choices)})" if choices else "") + ": ")

        print("Greetings! I'm your WeatherWise assistant, ready to bring you the latest forecast.")

        while True:
            while True:
                loc = ask("Enter the location (e.g., Perth)")
                if not loc.isdigit(): # Check if the input is not just digits
                    break
                else:
                    print(f"'{loc}' is not a valid location. Please enter a city or location name.")

            days = 5 # Fetch 5 days of data for potential future use
            data = get_weather_data(loc, forecast_days=days)

            _LAST_LOCATION, _LAST_FORECAST_DAYS, _LAST_WEATHER = loc, days, data

            if data and data.get("message") == "city not found":
                print(f"Error fetching weather data: {data['message']}. Please try another location.")
                # Continue the outer loop to ask for location again
            elif data and data.get("message"):
                print(f"Error fetching initial weather data: {data['message']}")
                break # Exit the loop for other errors
            elif not data:
                 print("Could not retrieve initial weather data for the specified location.")
                 # Optionally, break or continue based on desired behavior for other errors
                 break
            else:
                # Data fetched successfully, break the outer loop
                break


        while True:
                choices = [
                        "Fetch weather forecast",
                        "Show temperature chart",
                        # "Show precipitation chart",
                        "Show weather summary",
                        "Ask any question",
                        "Quit",
                ]
                choice = ask("Choose an option", choices)

                if choice == choices[0]:
                        if not _LAST_WEATHER:
                             loc = ask("Enter location (e.g., Perth)")
                             days = 5 # Fetch 5 days of data for potential future use
                             data = get_weather_data(loc, forecast_days=days)

                             _LAST_LOCATION, _LAST_FORECAST_DAYS, _LAST_WEATHER = loc, days, data

                             if data and data.get("message"):
                                 print(f"Error fetching weather data: {data['message']}")
                                 continue
                             elif not data:
                                 print("Could not retrieve weather data for the specified location.")
                                 continue


                        if _LAST_WEATHER and _LAST_WEATHER.get("list"):
                            daily_temps = get_temps(_LAST_WEATHER)
                            # Iterate through the first day's temperatures only
                            first_day_date = list(daily_temps.keys())[0]
                            temps = daily_temps[first_day_date]
                            average_temp = sum(temps) / len(temps) if temps else 0
                            print(f"\nAverage temperatures for {_LAST_LOCATION} Today ({first_day_date}) : {average_temp:.2f}°C")

                            # Add conditional messages based on average temperature
                            if 28 <= average_temp <= 32:
                                print("It is quite hot today. Please wear cool clothing and use sunscreen.")
                            elif 22 <= average_temp <= 27:
                                print("Today has good weather.")
                            elif 17 <= average_temp <= 21:
                                print("It is quite cold today. Please wear warm clothes if you go out.")
                            elif average_temp < 17:
                                print("It is freezing! Stay home and turn on the heater.")
                            elif average_temp > 32: # Corrected condition for "burning"
                                print("It is burning hot! Stay home and turn on the air conditioning.")
                        elif _LAST_WEATHER and _LAST_WEATHER.get("message"):
                             print(f"Error fetching weather data: {_LAST_WEATHER['message']}")
                        else:
                             print("Could not retrieve weather data for the specified location.")


                elif choice == choices[1]:
                        # Ask for the number of days for the chart
                        num_days_chart = pyip.inputInt("Enter the number of days for the temperature chart (up to 5): ", min=1, max=5)
                        create_temperature_visualisation(_LAST_WEATHER, num_days=num_days_chart, output_type='display')

                elif choice == choices[2]:
                        if not _LAST_WEATHER:
                                print("Please fetch weather first.")
                                continue
                        # TODO: implement weather summary function
                        create_weather_summary(_LAST_WEATHER)

                elif choice == choices[3]:
                        q = ask("Ask a weather question (e.g., 'Will it rain tomorrow in Sydney?')")
                        parsed = parse_weather_question(q)
                        # TODO : Edit function content
                        resp = generate_weather_response(parsed, _LAST_WEATHER)
                        print(resp)

                elif choice == choices[4]:
                        print("Goodbye!")
                        break

## 🧩 Main Application Logic

In [211]:
# Tie everything together here

def generate_weather_response(parsed_question: Dict[str, Any], weather_data: Dict[str, Any]) -> str:
	"""
	Generate a natural language response to a weather question.

	Args:
		parsed_question (dict): Parsed question data
		weather_data (dict): Weather data

	Returns:
		str: Natural language response
	"""
	if parsed_question.get("error"):
		return parsed_question["error"]
	if weather_data.get("error"):
		return weather_data["error"]

	location = weather_data.get("location") or parsed_question.get("location") or "the selected location"
	current = weather_data.get("current", {})
	daily = weather_data.get("daily", [])

	attribute = parsed_question.get("attribute")
	time_spec = parsed_question.get("time")

	def format_day(d: Dict[str, Any]) -> str:
		date = d.get("date", "")
		desc = (d.get("hourly", [{}])[0].get("weatherDesc") or [{"value": ""}])[0].get("value", "")
		return f"{date} ({desc})"

	if attribute == "temperature":
		# If today requested or unspecified, use current
		if time_spec in (None, 0):
			t = current.get("temp_C")
			feels = current.get("FeelsLikeC") or current.get("FeelsLikeC")
			desc = (current.get("weatherDesc") or [{"value": ""}])[0].get("value", "")
			if t is not None:
				return f"Right now in {location}: {t}°C, {desc}. Feels like {feels}°C."
		# Otherwise use daily forecast
		if isinstance(time_spec, int) and 0 <= time_spec < len(daily):
			d = daily[time_spec]
			return (
				f"In {location} on {d.get('date')}: min {d.get('mintempC')}°C, max {d.get('maxtempC')}°C."
			)
		if isinstance(time_spec, slice):
			parts = []
			for d in daily[time_spec]:
				parts.append(f"{d.get('date')}: {d.get('mintempC')}–{d.get('maxtempC')}°C")
			if parts:
				return f"Temperature forecast for {location}: " + "; ".join(parts)

	if attribute == "precipitation":
		if isinstance(time_spec, int) and 0 <= time_spec < len(daily):
			d = daily[time_spec]
			mm = d.get("totalPrecipMM")
			return f"Expected precipitation in {location} on {d.get('date')}: {mm} mm."
		if time_spec in (None, 0):
			# Try current precipitation
			precip_mm = current.get("precipMM")
			if precip_mm is not None:
				return f"Right now in {location}, precipitation is {precip_mm} mm."
		if isinstance(time_spec, slice):
			parts = []
			for d in daily[time_spec]:
				parts.append(f"{d.get('date')}: {d.get('totalPrecipMM', 0)} mm")
			if parts:
				return f"Precipitation forecast for {location}: " + "; ".join(parts)

	if attribute == "wind":
		if time_spec in (None, 0):
			spd = current.get("windspeedKmph")
			dirp = current.get("winddir16Point")
			if spd is not None:
				return f"Current wind in {location}: {spd} km/h {dirp or ''}."

	# Fallback generic summary
	if current:
		desc = (current.get("weatherDesc") or [{"value": ""}])[0].get("value", "")
		t = current.get("temp_C")
		return f"Currently in {location}: {t}°C, {desc}."

	return "I'm unable to summarise the weather from the provided data."

## 🧪 Testing and Examples

In [212]:
run_console_menu()

Greetings! I'm your WeatherWise assistant, ready to bring you the latest forecast.
Enter the location (e.g., Perth)5
'5' is not a valid location. Please enter a city or location name.
Enter the location (e.g., Perth)5
'5' is not a valid location. Please enter a city or location name.
Enter the location (e.g., Perth)Perth
Choose an option
1. Fetch weather forecast
2. Show temperature chart
3. Show weather summary
4. Ask any question
5. Quit
5
Goodbye!


## 🗂️ AI Prompting Log (Optional)
Add markdown cells here summarising prompts used or link to AI conversations in the `ai-conversations/` folder.

In [232]:
import ipywidgets as widgets
from IPython.display import display

# Create input widgets
location_input = widgets.Text(
    value=_LAST_LOCATION if _LAST_LOCATION else '',
    placeholder='Enter location (e.g., Perth)',
    description='Location:',
    disabled=False
)

days_input = widgets.IntText(
    value=_LAST_FORECAST_DAYS,
    description='Days (1-6):',
    disabled=False,
    min=1,
    max=6,
    layout=widgets.Layout(description_width='250 px') # Adjust description width
)

# Create action buttons
button_layout = widgets.Layout(width='250px') # Define a layout for buttons

fetch_button = widgets.Button(description="Fetch Weather Forecast", layout=button_layout)
temp_chart_button = widgets.Button(description="Show Temperature Chart", layout=button_layout)
precip_chart_button = widgets.Button(description="Show Precipitation Chart", layout=button_layout)
should_go_outside_button = widgets.Button(description="Should we go outside tomorrow?", layout=button_layout)
quit_button = widgets.Button(description="Quit", layout=button_layout)

# Create an output widget to display results
output_area = widgets.Output()

# Arrange widgets in a layout
input_widgets = widgets.VBox([location_input, days_input])
button_widgets = widgets.VBox([fetch_button, temp_chart_button, precip_chart_button, should_go_outside_button, quit_button]) # Removed summary_button from layout

ui = widgets.VBox([
    widgets.Label("Greetings! I'm your WeatherWise assistant, ready to bring you the latest forecast."),
    input_widgets,
    button_widgets,
    output_area
])

# Define button click handlers
def on_fetch_button_clicked(b):
    with output_area:
        output_area.clear_output()
        global _LAST_WEATHER, _LAST_LOCATION, _LAST_FORECAST_DAYS # Moved global declaration to the top
        loc = location_input.value
        # Fetch only today's weather for this button
        days = 6  # Fetch 6 days to potentially use for summary or other future features
        data = get_weather_data(loc, forecast_days=days)

        _LAST_WEATHER = data
        _LAST_LOCATION = loc
        _LAST_FORECAST_DAYS = days

        if data and data.get("list"):
            daily_temps = get_temps(data)
            if daily_temps:
                # Get current temperature (using the first available data point as a proxy)
                current_temp = data["list"][0]["main"]["temp"]
                current_description = data["list"][0]["weather"][0]["description"]
                humidity = data["list"][0]["main"]["humidity"]
                wind_speed = data["list"][0]["wind"]["speed"]

                print(f"Weather for {loc}:")
                print(f"Currently: {current_temp:.1f}°C with {current_description}, Humidity: {humidity}%, Wind: {wind_speed} m/s")

                # Get average temperature for the first day (today)
                first_day_date = list(daily_temps.keys())[0]
                temps = daily_temps[first_day_date]
                average_temp = sum(temps) / len(temps) if temps else 0

                print(f"Average temperature for Today ({first_day_date}): {average_temp:.2f}°C")

                # Add conditional messages based on average temperature
                if 28 <= average_temp <= 32:
                    print("Advice: It is quite hot today. Please wear cool clothing and use sunscreen.")
                elif 22 <= average_temp <= 27:
                    print("Advice: Today has good weather.")
                elif 17 <= average_temp <= 21:
                    print("Advice: It is quite cold today. Please wear warm clothes if you go out.")
                elif average_temp < 17:
                    print("Advice: It is freezing! Stay home and turn on the heater.")
                elif average_temp > 32: # Corrected condition for "burning"
                    print("Advice: It is burning hot! Stay home and turn on the air conditioning.")
            else:
                 print("Could not retrieve daily temperature data for the specified location.")


        elif data and data.get("message"):
             print(f"Error fetching weather data: {data['message']}")
        else:
             print("Could not retrieve weather data for the specified location.")


def on_temp_chart_button_clicked(b):
    with output_area:
        output_area.clear_output()
        global _LAST_WEATHER, _LAST_LOCATION, _LAST_FORECAST_DAYS # Moved global declaration to the top
        loc = location_input.value
        if not _LAST_WEATHER or _LAST_LOCATION != loc: # Fetch if no data or location changed
            print(f"Fetching weather data for {loc}...")
            days = 6
            data = get_weather_data(loc, forecast_days=days)
            _LAST_WEATHER = data
            _LAST_LOCATION = loc
            _LAST_FORECAST_DAYS = days
            if not data or data.get("message"):
                print("Could not retrieve weather data. Please check location.")
                return

        if not _LAST_WEATHER or not _LAST_WEATHER.get("list"):
            print("Weather data is not available to create chart. Please fetch weather first.")
            return

        num_days_chart = days_input.value
        create_temperature_visualisation(_LAST_WEATHER, num_days=num_days_chart, output_type='display')

def on_precip_chart_button_clicked(b):
    with output_area:
        output_area.clear_output()
        global _LAST_WEATHER, _LAST_LOCATION, _LAST_FORECAST_DAYS # Moved global declaration to the top
        loc = location_input.value
        if not _LAST_WEATHER or _LAST_LOCATION != loc: # Fetch if no data or location changed
            print(f"Fetching weather data for {loc}...")
            days = 6
            data = get_weather_data(loc, forecast_days=days)
            _LAST_WEATHER = data
            _LAST_LOCATION = loc
            _LAST_FORECAST_DAYS = days
            if not data or data.get("message"):
                print("Could not retrieve weather data. Please check location.")
                return

        if not _LAST_WEATHER or not _LAST_WEATHER.get("list"):
            print("Weather data is not available to create chart. Please fetch weather first.")
            return

        # The precipitation chart currently plots all available days (up to 6)
        create_precipitation_visualisation(_LAST_WEATHER, output_type='display')


def on_should_go_outside_button_clicked(b):
    with output_area:
        output_area.clear_output()
        global _LAST_WEATHER, _LAST_LOCATION, _LAST_FORECAST_DAYS
        loc = location_input.value

        if not _LAST_WEATHER or _LAST_LOCATION != loc: # Fetch if no data or location changed
            print(f"Fetching weather data for {loc}...")
            days = 6
            data = get_weather_data(loc, forecast_days=days)
            _LAST_WEATHER = data
            _LAST_LOCATION = loc
            _LAST_FORECAST_DAYS = days
            if not data or data.get("message"):
                print("Could not retrieve weather data. Please check location.")
                return

        if not _LAST_WEATHER or not _LAST_WEATHER.get("list"):
            print("Weather data is not available for tomorrow's forecast. Please fetch weather first.")
            return

        # Get tomorrow's data - assuming the first day in the list is today, the second is tomorrow
        if len(_LAST_WEATHER.get("list", [])) > 8: # OpenWeatherMap provides data in 3-hour intervals, so need at least 8*3 = 24 hours for tomorrow
            tomorrow_data_points = _LAST_WEATHER["list"][8:16] # Assuming data points 8-15 cover tomorrow
            if tomorrow_data_points:
                # Calculate average temperature for tomorrow
                tomorrow_temps = [item["main"]["temp"] for item in tomorrow_data_points]
                average_temp_tomorrow = sum(tomorrow_temps) / len(tomorrow_temps) if tomorrow_temps else 0

                # Calculate total precipitation for tomorrow (summing up 3-hour intervals)
                # OpenWeatherMap's forecast doesn't always have 'rain' or 'snow' keys, need to check
                tomorrow_precipitation = sum(item.get("rain", {}).get("3h", 0) + item.get("snow", {}).get("3h", 0) for item in tomorrow_data_points)

                # Get wind speed and description for tomorrow (using the first data point for simplicity)
                tomorrow_wind_speed = tomorrow_data_points[0]["wind"]["speed"]
                tomorrow_weather_description = tomorrow_data_points[0]["weather"][0]["description"]

                tomorrow_date = tomorrow_data_points[0]["dt_txt"].split(" ")[0]

                print(f"Tomorrow's Weather ({tomorrow_date}) for {loc}:")
                print(f"Average Temperature: {average_temp_tomorrow:.2f}°C")
                print(f"Total Precipitation: {tomorrow_precipitation:.2f} mm")
                print(f"Wind Speed: {tomorrow_wind_speed:.2f} m/s")
                print(f"Conditions: {tomorrow_weather_description}")


                # Provide advice based on tomorrow's weather
                print("\nShould we go outside tomorrow?")
                if tomorrow_precipitation > 0.5: # Threshold for significant rain
                    print("Advice: It looks like there will be significant rain tomorrow. It might be best to stay inside or be prepared for wet conditions.")
                elif average_temp_tomorrow < 10: # Threshold for cold
                     print("Advice: It will be quite cold tomorrow. Bundle up if you go outside!")
                elif average_temp_tomorrow > 30: # Threshold for hot
                     print("Advice: It will be quite hot tomorrow. Stay hydrated and seek shade if you go outside.")
                else:
                    print("Advice: The weather looks good for going outside tomorrow!")

            else:
                print("Could not retrieve detailed forecast data for tomorrow.")

        else:
             print("Insufficient forecast data to determine tomorrow's weather.")


def on_quit_button_clicked(b):
    with output_area:
        output_area.clear_output()
        print("Goodbye!")
    # In a real application, you might want to stop execution here
    # For a notebook, simply printing a message is sufficient


# Link buttons to handlers
fetch_button.on_click(on_fetch_button_clicked)
temp_chart_button.on_click(on_temp_chart_button_clicked)
# summary_button.on_click(on_summary_button_clicked) # Removed
precip_chart_button.on_click(on_precip_chart_button_clicked)
should_go_outside_button.on_click(on_should_go_outside_button_clicked)
quit_button.on_click(on_quit_button_clicked)


# Display the UI
display(ui)